In [1]:
import pandas as pd

from src.models import pooled_proba_model
from src.models import pooled_regression_model
from src.models import quote_regression_model

from src.features import data_for_modelling

# get data
orders_raw, offers_raw = data_for_modelling.get_raw_data()

# clean data
_, _, merged = data_for_modelling.clean_all(orders_raw, offers_raw)

# split offers into train test
merged_train, merged_test = data_for_modelling.split_train_test(merged)

# pickle data
merged_train.to_pickle("./data/pickels/merged_train.pkl") 
merged_test.to_pickle("./data/pickels/merged_test.pkl") 

# # read pickled data
# merged_train = pd.read_pickle("./data/pickels/merged_train.pkl") 
# merged_test = pd.read_pickle("./data/pickels/merged_test.pkl") 

In [9]:
pooled_proba_pl, (df_X_test, df_y_test) = pooled_proba_model.generate_probability_pipeline(merged_train)
pooled_reg_pl, (df_X_test, df_y_test) = pooled_regression_model.generate_pooled_regression_pipeline(merged_train)
quote_reg_pl, (df_X_test, df_y_test) = quote_regression_model.generate_quote_regression_pipeline(merged_train)

probas = pooled_proba_pl.predict_proba(merged_test).T[0]
pooled_cost = pooled_reg_pl.predict(merged_test)
quote_cost = quote_reg_pl.predict(merged_test)

weighted_cost = probas * pooled_cost  + (1-probas) *  quote_cost

In [ ]:
merged_test.sort_values(by=["ORDER_DATETIME_PST", "CREATED_ON_HQ"], ascending=True)